In [5]:
# dna_building_generator.ipynb - Cell 1: 匯入與設定

from compas_notebook.viewer import Viewer
from compas.geometry import Point, Vector, Polyline, Line
from compas.colors import Color
from compas.datastructures import Mesh
import math
import json
import os
import numpy as np
import pandas as pd

# ----------------------------------------------------
# 1. 自訂建築參數 
# ----------------------------------------------------
CUSTOM_RADIUS = 1.50                # 建築物的半徑 (影響長寬)
CUSTOM_HEIGHT = 10            # 建築物的總高度
CUSTOM_FLOORS = 20                  # 總樓層數
FLOOR_HEIGHT = CUSTOM_HEIGHT / CUSTOM_FLOORS 
RUNG_WIDTH = CUSTOM_RADIUS * 0.15   # 樓板厚度/寬度

R = CUSTOM_RADIUS                       # 用於 CFD 核心的建築半徑
HEIGHT = CUSTOM_HEIGHT                  # 用於 CFD 核心的建築高度 

# ----------------------------------------------------
# 2. 匯入最佳扭轉角
# ----------------------------------------------------
imported_angle = 0.0 # 預設值，以防匯入失敗

try:
    with open('best_angle_data.json', 'r') as f:
        data = json.load(f)
        imported_angle = data.get('best_angle_deg', 0.0)
        
    print(f"✅ 已成功匯入最佳扭轉角: {imported_angle:.2f} 度")

except FileNotFoundError:
    print("❌ 錯誤：找不到 best_angle_data.json 檔案。請先運行 wind_analysis.ipynb 的匯出步驟。")
    print("   使用預設角度 0.0 度。")
except Exception as e:
    print(f"❌ 匯入時發生錯誤: {e}")
    print("   使用預設角度 0.0 度。")
    
# 最終使用的扭轉角
BEST_TWIST_ANGLE = imported_angle

# ----------------------------------------------------
# 3. CFD
# ----------------------------------------------------
WIND_DIR = Vector(1, 0, 0)      # 主風向（+X）
U_INF = 1.0                     # 來流風速
R_OBS = R * 1.15                # DNＡ 視為障礙物的等效半徑
ALPHA = 1.8                     # 繞流強度

# 模擬領域範圍
N_Y, N_Z = 3, 3                 # 模擬流線的數量 (密度)
Y_RANGE = (-2.2 * R, 2.2 * R)   
Z_RANGE = (0.0, HEIGHT)         

# 積分步長與繪圖參數
STREAM_LEN = 6.0 * R            
STEP_SIZE  = 0.08 * R           
SEG_LEN    = 0.20 * R           
GAP_LEN    = 0.08 * R           
HEAD_SCALE = 0.25               
X_START = -5.0 * R              

# 顏色設定
COLOR_SLOW = Color(0.2, 0.6, 1.0, 0.9) 
COLOR_FAST = Color(1.0, 0.3, 0.2, 0.9) 

# 全域變數 (用於儲存 CFD 結果)
segments = []      
wind_vectors = [] 
MAX_OBJECTS = 8000
_obj_count = 0


# ----------------------------------------------------
# 4. Viewer 初始化
# ----------------------------------------------------
viewer = Viewer()
viewer.scene.clear()

✅ 已成功匯入最佳扭轉角: 90.00 度


In [6]:
# dna_building_analysis.ipynb - Cell 2: CFD 核心與單次分析函式

def swirl_around_DNA(p: Point) -> Vector:
    """產生繞著 DNA 中軸的旋流速度分量。"""
    # ... (此處邏輯保持不變，略)
    cy = 0.5 * (Y_RANGE[0] + Y_RANGE[1])
    cz = 0.5 * (Z_RANGE[0] + Z_RANGE[1])
    dy = p.y - cy
    dz = p.z - cz
    r2 = dy * dy + dz * dz
    if r2 < 1e-12: return Vector(0.0, 0.0, 0.0)
    r = math.sqrt(r2)
    t_y = -dz / r
    t_z =  dy / r
    base = ALPHA * (R_OBS ** 2 / max(r2, R_OBS ** 2))
    if r < R: base *= 1.5
    return Vector(0.0, base * t_y, base * t_z)

def velocity_field(p: Point) -> Vector:
    v = WIND_DIR.unitized() * U_INF
    cy, cz = 0.5 * (Y_RANGE[0] + Y_RANGE[1]), 0.5 * (Z_RANGE[0] + Z_RANGE[1])
    dy, dz = p.y - cy, p.z - cz
    r = math.sqrt(dy*dy + dz*dz)
    swirl = swirl_around_DNA(p)
    v = v + swirl
    if r < R_OBS: v = v * (r / R_OBS)**2 
    if p.x > 0: v = v * math.exp(-(r / (1.5 * R)))
    return v

def rk2(p: Point, h: float) -> Point:
    # ... (此處邏輯保持不變，略)
    k1 = velocity_field(p)
    mid = p + k1 * (0.5 * h)
    k2 = velocity_field(mid)
    return p + k2 * h

def analyze_and_generate_flow():
    global segments, wind_vectors 
    segments.clear()
    wind_vectors.clear()
    
    # --- 流線積分邏輯 (保持不變) ---
    y0, y1 = Y_RANGE
    z0, z1 = Z_RANGE
    zc_mid = 0.5 * (Z_RANGE[0] + Z_RANGE[1])
    
    for iy in range(N_Y):
        y = y0 + (y1 - y0) * (iy + 0.5) / N_Y
        for iz in range(N_Z):
            z = z0 + (z1 - z0) * (iz + 0.5) / N_Z
            cur = Point(X_START, y, z)
            total = 0.0
            carry = 0.0
            seg_start = cur
            
            while total < STREAM_LEN:
                nxt = rk2(cur, STEP_SIZE)
                step_d = Vector.from_start_end(cur, nxt).length
                
                v = velocity_field(cur)
                wind_vectors.append(v)

                if (cur.y ** 2 + (cur.z - zc_mid) ** 2) < (0.9 * R_OBS) ** 2:
                    phi = math.atan2(cur.z - zc_mid, cur.y)
                    push = Vector(-math.sin(phi), math.cos(phi), 0.0) * (0.05 * R)
                    cur = cur + push
                    continue

                carry += step_d
                total += step_d
                cur = nxt

                if carry >= SEG_LEN:
                    segments.append((seg_start, cur, v.length))
                    dirv = Vector.from_start_end(seg_start, cur)
                    if dirv.length > 1e-9:
                        diru = dirv.unitized()
                        seg_start = cur + diru * GAP_LEN
                        cur = seg_start
                    carry = 0.0
    # --------------------------------

    pass # 函式執行完畢後，segments 和 wind_vectors 已經被填充

In [7]:
# dna_building_analysis.ipynb - Cell 3: 幾何與視覺化輔助函式

# ----------------- 幾何核心函式 -----------------

def quad_at_center_and_dir(center: Point, u_dir: Vector, length: float, width: float):
    u = u_dir.copy()
    if u.length == 0: u = Vector(1, 0, 0)
    u.unitize()
    v = Vector(0, 0, 1).cross(u)
    if v.length == 0: v = Vector(1, 0, 0).cross(u)
    v.unitize()
    halfL, halfW = 0.5 * length, 0.5 * width
    a = center + (u * halfL) + (v * halfW)
    b = center - (u * halfL) + (v * halfW)
    c = center - (u * halfL) - (v * halfW)
    d = center + (u * halfL) - (v * halfW)
    return [a, b, c, d]

def prism_from_quad(quad, height):
    lift = Vector(0, 0, height)
    a, b, c, d = quad
    a2, b2, c2, d2 = a + lift, b + lift, c + lift, d + lift
    vertices = [a, b, c, d, a2, b2, c2, d2]
    faces = [[0, 1, 2, 3], [4, 5, 6, 7], [0, 1, 5, 4], [1, 2, 6, 5], [2, 3, 7, 6], [3, 0, 4, 7]]
    return Mesh.from_vertices_and_faces(vertices, faces)

def create_twisted_dna_mesh(total_angle_deg: float, R: float, total_floors: int, floor_height: float, rung_width: float) -> list[Mesh]:
    
    total_angle_rad = math.radians(total_angle_deg) 
    angle_per_floor = total_angle_rad / (total_floors - 1) if total_floors > 1 else 0

    meshes = []
    
    for n in range(total_floors):
        t_current = n * angle_per_floor
        zc = n * floor_height
        extrude_height = floor_height
        
        p1 = Point(R * math.cos(t_current),  R * math.sin(t_current),  zc)
        p2 = Point(-R * math.cos(t_current), -R * math.sin(t_current), zc)
        center = Point(0, 0, zc)
        
        u_dir = Vector.from_start_end(p1, p2)
        length = (p1 - p2).length
        
        quad = quad_at_center_and_dir(center, u_dir, length, rung_width)
        solid = prism_from_quad(quad, extrude_height)
        meshes.append(solid)
        
    return meshes

def helix_points(R, total_height, total_floors, phase=0.0):
    
    points_per_strand = total_floors * 10 
    pts = []
    
    # ⚠️ 從 Cell 1 的全域變數中讀取扭轉角
    total_twists_rad = math.radians(BEST_TWIST_ANGLE)
    
    for i in range(points_per_strand + 1):
        t = total_twists_rad * (i / points_per_strand) + phase
        z = total_height * (i / points_per_strand)
        
        pts.append(Point(R * math.cos(t), R * math.sin(t), z))
    return pts
    
# ----------------- 視覺化輔助函式 -----------------

def draw_arrow(p0: Point, p1: Point, col: Color, viewer: Viewer):
    global _obj_count
    if _obj_count >= MAX_OBJECTS: return

    viewer.scene.add(Line(p0, p1), color=col)
    _obj_count += 1

    dirv = Vector.from_start_end(p0, p1)
    L = dirv.length
    if L < 1e-9: return
    u = dirv.unitized()
    
    v = Vector(0, 0, 1).cross(u)
    if v.length < 1e-9: v = Vector(0, 1, 0).cross(u)
    v.unitize()

    head_len = HEAD_SCALE * L 
    head_wid = 0.5 * HEAD_SCALE * L
    tip  = p1
    base = p1 - u * head_len
    left = base + v * head_wid
    right = base - v * head_wid

    viewer.scene.add(Line(left, tip),  color=col)
    viewer.scene.add(Line(right, tip), color=col)
    _obj_count += 2

def lerp_color(c0, c1, t):
    return Color(
        c0.r + (c1.r - c0.r) * t,
        c0.g + (c1.g - c0.g) * t,
        c0.b + (c1.b - c0.b) * t,
        c0.a + (c1.a - c0.a) * t,
    )

In [8]:
# dna_building_analysis.ipynb - Cell 4: 執行單次 CFD 分析與視覺化

viewer.scene.clear()

print(f"   開始生成 {CUSTOM_FLOORS} 層 DNA 建築...")
print(f"   總高度: {CUSTOM_HEIGHT} m")
print(f"   底面直徑: {CUSTOM_RADIUS * 2.0} m")
print(f"   總扭轉角: {BEST_TWIST_ANGLE:.2f} 度 (來自 wind_analysis.ipynb)")

# -----------------------------------------------
# 1. 運行單次 CFD 分析並生成流線數據
# -----------------------------------------------
analyze_and_generate_flow() 
print(f"\n 對此自訂建築進行 CFD 模擬，流線數據已生成。")

# -----------------------------------------------
# 2. 創建並添加 DNA 建築 Mesh (保持不變)
# -----------------------------------------------
dna_meshes = create_twisted_dna_mesh(
    total_angle_deg=BEST_TWIST_ANGLE, 
    R=CUSTOM_RADIUS, 
    total_floors=CUSTOM_FLOORS, 
    floor_height=FLOOR_HEIGHT, 
    rung_width=RUNG_WIDTH
)

for i, mesh in enumerate(dna_meshes):
    col = Color(0.60, 0.60, 0.65) if (i % 2 == 0) else Color(0.40, 0.40, 0.45)
    viewer.scene.add(mesh, color=col)
    
# 3. 繪製 DNA 雙股螺旋線 (保持不變)
pts1 = helix_points(CUSTOM_RADIUS, CUSTOM_HEIGHT, CUSTOM_FLOORS, phase=0.0)
pts2 = helix_points(CUSTOM_RADIUS, CUSTOM_HEIGHT, CUSTOM_FLOORS, phase=math.pi)

poly1 = Polyline(pts1)
poly2 = Polyline(pts2)

viewer.scene.add(poly1, name="strand_A", color=Color(0.15, 0.45, 1.0), linewidth=5)
viewer.scene.add(poly2, name="strand_B", color=Color(1.0, 0.25, 0.25), linewidth=5)

# -----------------------------------------------
# 4. 繪製 CFD 流線 (使用 segments 數據)
# -----------------------------------------------
if segments:
    speeds = [s for _, _, s in segments]
    s_min, s_max = min(speeds), max(speeds)
    if abs(s_max - s_min) < 1e-8: s_max = s_min + 1.0

    for p0, p1, s in segments:
        t = (s - s_min) / (s_max - s_min)
        col = lerp_color(COLOR_SLOW, COLOR_FAST, t) 
        draw_arrow(p0, p1, col, viewer)
        
# 5. 顯示結果
viewer.show()

print("\n✅ 自訂 DNA 建築與 CFD 流線視覺化完成。")

   開始生成 20 層 DNA 建築...
   總高度: 10 m
   底面直徑: 3.0 m
   總扭轉角: 90.00 度 (來自 wind_analysis.ipynb)

 對此自訂建築進行 CFD 模擬，流線數據已生成。



✅ 自訂 DNA 建築與 CFD 流線視覺化完成。
